In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

def load_model_and_weights(model_weights_path):
    # Load the base model
    base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Freeze the base model
    base_model.trainable = False
    
    # Add a global average pooling layer
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    
    # Add a dense layer with 11 units and softmax activation for classification
    output = tf.keras.layers.Dense(11, activation='softmax')(x)
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=output)
    
    # Load the trained weights
    model.load_weights(model_weights_path)
    
    return model

def predict_class(model, img_path, class_names):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale pixel values to [0, 1]
    
    # Predict the class
    prediction = model.predict(img_array)
    
    # Get the predicted class label
    predicted_class = np.argmax(prediction)
    
    # Get the class name from the class_names list
    class_name = class_names[predicted_class]
    
    return class_name

def calculate_calories(predicted_class, total_volume, class_densities, calories_per_100g):
    # Calculate the total weight based on density and total volume
    total_weight = total_volume * class_densities[predicted_class]
    
    # Calculate the calories based on total weight
    calories = (total_weight / 100) * calories_per_100g[predicted_class]
    
    return calories

# Define the class names
class_names = ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles/Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable/Fruit']

# Define the densities corresponding to each class name
class_densities = {
    'Bread': 0.75,
    'Dairy product': 0.6,
    'Dessert': 0.8,
    'Egg': 0.65,
    'Fried food': 0.85,
    'Meat': 0.9,
    'Noodles/Pasta': 0.7,
    'Rice': 0.72,
    'Seafood': 0.88,
    'Soup': 0.78,
    'Vegetable/Fruit': 0.5
}

# Dictionary mapping class names to calories per 100g
calories_per_100g = {
    'Bread': 265,
    'Dairy product': 150,
    'Dessert': 250,
    'Egg': 155,
    'Fried food': 330,
    'Meat': 250,
    'Noodles/Pasta': 138,
    'Rice': 130,
    'Seafood': 85,
    'Soup': 30,
    'Vegetable/Fruit': 30
}

# Example usage
model_weights_path = "../../models/classification/inception_resnet_v2_classifier.h5"
image_path = "../../assets/readme_assets/examples/egg_plate.webp"
total_volume = 200  # in ml

loaded_model = load_model_and_weights(model_weights_path)
predicted_class = predict_class(loaded_model, image_path, class_names)
total_calories = calculate_calories(predicted_class, total_volume, class_densities, calories_per_100g)

print("Predicted class:", predicted_class)
print("Total calories in predicted class:", total_calories)


Predicted class: Egg
Total calories in predicted class: 201.5
